In [ ]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
config = {
    'n_epochs':         200, 
    'batch_size':       64,
    'learning_rate':    0.0002,
    'momentum_gradient': {
        'b1':           0.5,
        'b2':           0.999
    },
    'n_workers':        8,
    'latent_dim':       100, 
    'img_size':         28,
    'channels':         1,
    'sample_interval':   400
}

img_shape = (config['channels'], config['img_size'], config['img_size'])

In [ ]:
os.makedirs("images", exist_ok=True)

# Data processing

In [ ]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(), 
    transforms.Normalize([0.5], [0.5])
])

dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transform,),
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=config['n_workers']
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# Model Construction

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(config['latent_dim'], 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

# Training and Result

In [ ]:
# Initialize the Generator and the Discriminator
generator = Generator()
discriminator = Discriminator() 

# Lost function
adversarial_loss = torch.nn.BCELoss()

# Optimizer
optimizer_G = torch.optim.Adam(generator.parameters(), lr=config['learning_rate'],
                         betas=(config['momentum_gradient']['b1'], config['momentum_gradient']['b2']))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=config['learning_rate'],
                         betas=(config['momentum_gradient']['b1'], config['momentum_gradient']['b2']))

# Device and Tensor type
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} for training !!!'.format(device))
Tensor = torch.cuda.FloatTensor if 'cuda' else torch.FloatTensor

Using cuda for training !!!


In [ ]:
generator.to(device)

Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=784, bias=True)
    (12): Tanh()
  )
)

In [ ]:
discriminator.to(device)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [ ]:
for epoch in range(config['n_epochs']):
  for i, (imgs, _) in enumerate(dataloader):
    #Adversarial ground truths
    valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
    fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

    # Input
    real_imgs = Variable(imgs.type(Tensor))

    # Generator training
    optimizer_G.zero_grad()
    z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], config['latent_dim']))))
    gen_imgs = generator(z)
    loss_G = adversarial_loss(discriminator(gen_imgs), valid)
    loss_G.backward()
    optimizer_G.step()

    # Discriminator training
    optimizer_D.zero_grad()
    real_loss = adversarial_loss(discriminator(real_imgs), valid)
    fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
    loss_D = (real_loss + fake_loss)/2
    loss_D.backward()
    optimizer_D.step()

    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, config['n_epochs'], i, len(dataloader), loss_D.item(), loss_G.item())
    )

    batches_done = epoch * len(dataloader) + i
    if batches_done % config['sample_interval'] == 0:
        save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)


Streaming output truncated to the last 5000 lines.
[Epoch 194/200] [Batch 628/938] [D loss: 0.108770] [G loss: 3.428672]
[Epoch 194/200] [Batch 629/938] [D loss: 0.242018] [G loss: 2.700352]
[Epoch 194/200] [Batch 630/938] [D loss: 0.258073] [G loss: 3.319502]
[Epoch 194/200] [Batch 631/938] [D loss: 0.277062] [G loss: 1.563177]
[Epoch 194/200] [Batch 632/938] [D loss: 0.287959] [G loss: 7.109627]
[Epoch 194/200] [Batch 633/938] [D loss: 0.148525] [G loss: 4.875983]
[Epoch 194/200] [Batch 634/938] [D loss: 0.239327] [G loss: 2.656028]
[Epoch 194/200] [Batch 635/938] [D loss: 0.126546] [G loss: 3.382388]
[Epoch 194/200] [Batch 636/938] [D loss: 0.165289] [G loss: 4.241245]
[Epoch 194/200] [Batch 637/938] [D loss: 0.068792] [G loss: 3.017666]
[Epoch 194/200] [Batch 638/938] [D loss: 0.142864] [G loss: 3.307831]
[Epoch 194/200] [Batch 639/938] [D loss: 0.129461] [G loss: 2.453904]
[Epoch 194/200] [Batch 640/938] [D loss: 0.074723] [G loss: 4.382364]
[Epoch 194/200] [Batch 641/938] [D loss

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/images' '/content/drive/MyDrive/Dataset'